# Step 1: Convert to TensorFlow Lite Model

In [10]:
import tensorflow as tf

In [11]:
# Image parameters (match training)
IMG_SIZE = (96, 96)
BATCH_SIZE = 32

In [12]:
# Load a small validation set for quantization calibration
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    r"C:\Users\nisal\Downloads\IoT\ClassificationModel\dataset\val", 
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary",
    shuffle=False
)

Found 775 files belonging to 2 classes.


In [13]:
# Normalize like during training
normalization_layer = tf.keras.layers.Rescaling(1./255)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [14]:
# Load the saved model (SavedModel format)
model = tf.keras.models.load_model(r"C:\Users\nisal\Downloads\IoT\ClassificationModel\train_classifier_model")

In [15]:
# Create converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Quantization (Post-training integer quantization)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# If you want full integer quantization, you need a representative dataset
# We'll define a dummy function here as an example:
def representative_data_gen():
    for images, _ in val_ds.take(100):  # Use a small batch from validation set
        yield [images]

converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

In [16]:
# Convert the model
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\nisal\AppData\Local\Temp\tmpc11ew6uu\assets


INFO:tensorflow:Assets written to: C:\Users\nisal\AppData\Local\Temp\tmpc11ew6uu\assets
d:\Programs\Python\Python38\lib\site-packages\tensorflow\lite\python\convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [17]:
# Save it
with open("train_classifier_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved!")

TFLite model saved!


# Step 2: Convert .tflite to .h Header File - Using Python (cross-platform)

In [18]:
def convert_to_header(tflite_model_path, header_path, variable_name="train_classifier_model"):
    with open(tflite_model_path, "rb") as f:
        data = f.read()

    with open(header_path, "w") as f:
        f.write(f"const unsigned char {variable_name}[] = {{\n")
        for i, byte in enumerate(data):
            if i % 12 == 0:
                f.write("\n ")
            f.write(f"0x{byte:02x}, ")
        f.write(f"\n}};\n")
        f.write(f"const unsigned int {variable_name}_len = {len(data)};\n")

convert_to_header("train_classifier_model.tflite", "train_classifier_model.h")
